In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

In [3]:
PATH_TRAIN_X = "../UCI_HAR_Dataset/train/X_train.txt"
PATH_TRAIN_Y = "../UCI_HAR_Dataset/train/y_train.txt"

PATH_TEST_X = "../UCI_HAR_Dataset/test/X_test.txt"
PATH_TEST_Y = "../UCI_HAR_Dataset/test/y_test.txt"

PATH_TRAIN_SBJ = "../UCI_HAR_Dataset/train/subject_train.txt"

PATH_TEST_SBJ = "../UCI_HAR_Dataset/test/subject_test.txt"

PATH_FT = "../UCI_HAR_Dataset/features.txt"
features = pd.read_csv(PATH_FT, sep=" ", header=None, index_col=0).reset_index()

In [4]:
df_x_train = pd.read_fwf(PATH_TRAIN_X, header=None)
df_x_train.rename(columns=features[1], inplace=True)

y_train_col = pd.read_fwf(PATH_TRAIN_Y, header=None)

In [5]:
train_array = np.array(df_x_train)
len(train_array)

7352

In [6]:
model = tf.keras.Sequential([
    layers.Dense(7, activation="linear")
])
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [7]:
model.fit(train_array, y_train_col, epochs=3)

Epoch 1/3


In [ ]:
x_test = np.array(pd.read_fwf(PATH_TEST_X, header=None))
y_test = pd.read_fwf(PATH_TEST_Y, header=None)
model.fit(x_test, y_test, epochs=8)

### Evaluation

`model.evaluate(x_test,  y_test, verbose=2)`

`probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])`

`probability_model(x_test[:5])`